<a href="https://colab.research.google.com/github/Astenik/Astenik/blob/main/Inerview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install SpeechRecognition pydub
!pip install gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [ ]:
from openai import OpenAI
import os
import speech_recognition as sr
from gtts import gTTS
import IPython.display as ipd
from google.colab import files
import json
from datetime import datetime

In [ ]:
recognizer = sr.Recognizer()

os.environ["OPENAI_API_KEY"] = "sk-...-ast-...-h"
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

questions = [
    ("Full name/Background", "What is your full name and background?"),
    ("Expectations", "Why are you interested in joining the program?"),
    ("Experience", "What’s your experience with data science or AI?"),
    ("Goals", "What are your short-term and long-term goals?"),
    ("Availability", "Are you ready to start immediately? If not, when?")
]

unsertain_words = ["may be", "dont know"]

def needs_clarification(answer: str) -> bool:
  answer = answer.lower().strip()
  return len(answer) < 5 or any(u in answer for u in unsertain_words)

def rephrase_question(question: str) -> str:
  resp = client.chat.completions.create(
      model = "gpt-4",
      messages = [
          {"role": "system", "content": "You are an admission interviewer"},
          {"role": "user", "content": f"rephrase this question simplier: {question}"}
      ]
  )
  return resp.choices[0].message.content

def speak(text):
    tts = gTTS(text=text, lang="en")
    tts.save("temp.mp3")
    return ipd.Audio("temp.mp3")

def summarize_answers(answers: dict) -> str:
  prompt = f"""
  Here are the candidate's answers {answers}

  Summarize them in a structured way.
  Extract fields:
  -Full name/Background
  -Expectations
  -Experience
  -Goals
  -Availability

  return the result in JSON format
  """
  resp = client.chat.completions.create(
      model = "gpt-4",
      messages = [
          {"role": "system", "content": "You are an assistant that summarizes interview answers."},
          {"role": "user", "content": prompt}
      ]
  )
  return resp.choices[0].message.content

def save_session(answers: dict, summary: str, filename="session_log.json"):
    log = {
        "timestamp": datetime.now().isoformat(),
        "answers": answers,
        "summary": summary
    }
    with open(filename, "w") as f:
        json.dump(log, f, indent=4)
    print(f"Session saved to {filename}")

def handle_faq_query(user_question: str) -> str:
    faq_context = """
    Q: What’s the cost of the program?
    A: The program costs $500.

    Q: Do I need a technical background?
    A: A technical background is not required, but some familiarity with math and statistics is helpful.

    Q: Can I do this part-time?
    A: Yes, the program is designed to be flexible and can be completed part-time.
    """
    prompt = f"""
    Answer the following question based on the provided FAQ context. If the answer is not in the context, state that you do not have the information.

    FAQ Context: {faq_context}

    User's Question: {user_question}

    Answer:
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant for the LunarTech bootcamp."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

def main():

  #Part 1: Conversational Flow
  answers = {}

  for key, q in questions:
    current_question = q

    for attempt in range(2):
      display(speak(current_question))
      user_ans = ''
      print("Please record your answer and upload here.")

      uploaded = files.upload()

      audio_file = list(uploaded.keys())[0]

      with sr.AudioFile(audio_file) as source:
        audio = recognizer.record(source)

        try:
          user_ans = recognizer.recognize_google(audio, language = "en-US")

        except sr.UnknownValueError:
          display(speak("Sorry, could not understand audio."))

        except sr.RequestError:
          display(speak("API request error. Please check your internet connection and API key."))

      if not needs_clarification(user_ans):
        answers[key] = user_ans
        break
      else:
        current_question = rephrase_question(q)
        if attempt == 1:
          answers[key] = user_ans

  #Part 2: Q&A Capability
  display(speak("Now, do you have any questions for me about the program?"))
    while True:
        print("Please record your question or say 'no' to finish.")
        uploaded = files.upload()
        audio_file = list(uploaded.keys())[0]

        with sr.AudioFile(audio_file) as source:
            audio = recognizer.record(source)
            try:
                user_question = recognizer.recognize_google(audio, language="en-US").lower()
            except (sr.UnknownValueError, sr.RequestError):
                display(speak("Sorry, I didn't catch that. Do you have a question?"))
                continue

        if 'no' in user_question or 'no thanks' in user_question:
            display(speak("Okay, thank you for your time!"))
            break

        faq_response = handle_faq_query(user_question)
        display(speak(faq_response))


  summary = summarize_answers(answers)
  print("\n--- Interview summary ---")
  print(summary)

  save_session(answers, summary)

if __name__ == "__main__":
  main()